In [1]:
import os
from pathlib import Path

# change working directory to make src visible
os.chdir(Path.cwd().parent)

In [2]:
import numpy as np
from src.tracknet.visualization import visualize_track_predictions

# Generate example batch
example_batch = {
    # One track with 3 hits
    'inputs': np.array([[[0, 0, 0], [5, 8, 3], [12, 15, 6]]]),
    'targets': np.array([[[5, 8, 3], [12, 15, 6], [18, 22, 9]]]),
    'input_lengths': [3],
    'target_mask': np.array([[True, True, True]])
}

example_output = {
    'coords_t1': np.array([[[6, 7, 3], [11, 16, 6], [17, 23, 9]]]),
    'radius_t1': np.array([[[2], [2], [2]]]),
    'coords_t2': np.array([[[11, 15, 6], [17, 22, 9], [24, 28, 12]]]),
    'radius_t2': np.array([[[3], [3], [3]]])
}


# Create and show visualization
fig = visualize_track_predictions(example_batch, example_output)
fig.show()

## Use Randomly initialized model and real tracks

In [3]:
from torch.utils.data import DataLoader
from src.tracknet.data.dataset import TrackMLTracksDataset
from src.tracknet.data.transformations import MinMaxNormalizeXYZ, DropRepeatedLayerHits
from src.tracknet.data.filters import MinHitsFilter, PtFilter, FirstLayerFilter
from src.tracknet.data.collate import collate_fn

BATCH_SIZE = 4

val_dataset = TrackMLTracksDataset(
    data_dirs=Path("data/trackml/train_100_events"),
    blacklist_dir=Path("data/trackml/blacklist_training"),
    validation_split=0.1,
    split="validation",
    transforms=[
        DropRepeatedLayerHits(),
        MinMaxNormalizeXYZ(
            min_xyz=(-1000.0, -1000.0, -3000.0),
            max_xyz=(1000.0, 1000.0, 3000.0)
        )
    ],
    filters=[
        MinHitsFilter(min_hits=3),
        PtFilter(min_pt=1.0),
        FirstLayerFilter(
            {(8, 2), (7, 14), (9, 2)}
        )
    ]
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    collate_fn=collate_fn,
    num_workers=4,
    persistent_workers=True,
)

In [13]:
import torch

from src.tracknet.model import StepAheadTrackNET
from src.tracknet.metrics import SearchAreaMetric, HitEfficiencyMetric
from src.tracknet.visualization import visualize_track_predictions


model = StepAheadTrackNET()
t1_area_metric = SearchAreaMetric("t1")
t2_area_metric = SearchAreaMetric("t2")
t1_efficiency = HitEfficiencyMetric("t1")
t2_efficiency = HitEfficiencyMetric("t2")

for batch in val_loader:
    with torch.no_grad():
        outputs = model(batch["inputs"], batch["input_lengths"])
    # take only a single sample for visualization and metrics
    i = 1
    one_output = {k: v[i:i + 1] for k, v in outputs.items()}
    one_sample = {k: v[i:i + 1] for k, v in batch.items()}

    # calculate metrics
    t1_area = t1_area_metric(one_output, one_sample["target_mask"])
    t2_area = t2_area_metric(one_output, one_sample["target_mask"])
    t1_eff = t1_efficiency(
        one_output, one_sample["targets"], one_sample["target_mask"])
    t2_eff = t2_efficiency(
        one_output, one_sample["targets"], one_sample["target_mask"])
    print(f"Metrics:\n\tt1_area: {t1_area}, t1_eff: {t1_eff}\n\t"
          f"t2_area: {t2_area}, t2_eff: {t2_eff}")
    break

visualize_track_predictions(batch, outputs, track_idx=i).show()

Metrics:
	t1_area: 1.6572095155715942, t1_eff: 0.3333333432674408
	t2_area: 1.274411678314209, t2_eff: 0.0
